Installing the dependencies

In [8]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=255319 sha256=356d859366d2b7ccf794509ecb964bec1e0d6ce4309b3328e990cb1caf108f02
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.2/127.2 kB 7.6 MB/s eta 0:00:00


In [19]:
from fastapi import FastAPI,HTTPException
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

In [10]:
app = FastAPI()

In [11]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification

# Define the number of samples
num_samples = 1000

# Define the number of features
num_features = 8  # Assuming the same features as in your sample data

# Generate synthetic data using scikit-learn's make_classification
X_synthetic, y_synthetic = make_classification(n_samples=num_samples, n_features=num_features, n_classes=2, random_state=42)

# Convert the NumPy arrays into a pandas DataFrame
synthetic_data = pd.DataFrame(X_synthetic, columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'])
synthetic_data['Outcome'] = y_synthetic

# Display the synthetic data
print(synthetic_data.head())

# Save the synthetic data to a CSV file
synthetic_data.to_csv('synthetic_data.csv', index=False)


   Pregnancies   Glucose  BloodPressure  SkinThickness   Insulin       BMI  \
0     0.538435 -1.236136       0.212184       0.174502 -0.277035  1.254338   
1    -0.376301 -2.929449      -0.144079       0.023662  0.652572  2.052972   
2     0.155191  0.000207       0.531307       0.371714 -0.906845 -0.009300   
3    -0.237683  1.598322       0.394418       0.648727  0.544798  0.733548   
4    -0.501402 -1.002217      -0.226625      -0.410414 -0.436103 -0.752435   

   DiabetesPedigreeFunction       Age  Outcome  
0                 -0.225485  0.731001        1  
1                 -0.281049  1.088770        1  
2                 -0.365368 -0.327895        0  
3                 -1.409775  0.008784        0  
4                  0.925074 -1.466785        1  


Build ML application


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split # test and train
from sklearn.preprocessing import StandardScaler # Normalisation
from sklearn.linear_model import LogisticRegression # Model
import joblib #
data = pd.read_csv('synthetic_data.csv')


In [3]:
# Split the features and target variable
X = data[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']]
y = data['Outcome']

train and test split

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [7]:
# Create and train the logistic regression model
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# Evaluate the model
accuracy = model.score(X_test_scaled, y_test)
print("Accuracy:", accuracy)

# Save the trained model
joblib.dump(model, 'diabetes_model.sav')

Accuracy: 0.875


['diabetes_model.sav']

Load the model

In [18]:
import joblib
diabetes_model = joblib.load('diabetes_model.sav')


In [20]:
# Define the input data schema using Pydantic BaseModel
class ModelInput(BaseModel):
    Pregnancies: float
    Glucose: float
    BloodPressure: float
    SkinThickness: float
    Insulin: float
    BMI: float
    DiabetesPedigreeFunction: float
    Age: float

app = FastAPI()

# Define the input data schema using Pydantic BaseModel
class ModelInput(BaseModel):
    Pregnancies: float
    Glucose: float
    BloodPressure: float
    SkinThickness: float
    Insulin: float
    BMI: float
    DiabetesPedigreeFunction: float
    Age: float

app = FastAPI()

@app.post('/diabetes_prediction')
def diabetes_pred(input_data: ModelInput):
    try:
        input_list = [input_data.Pregnancies, input_data.Glucose, input_data.BloodPressure,
                      input_data.SkinThickness, input_data.Insulin, input_data.BMI,
                      input_data.DiabetesPedigreeFunction, input_data.Age]

        # Convert input list to numpy array and reshape
        input_array = np.array(input_list).reshape(1, -1)

        # Make prediction
        prediction = diabetes_model.predict(input_array)

        # Interpret prediction
        if prediction[0] == 0:
            return {'prediction': 'The person is not diabetic'}
        else:
            return {'prediction': 'The person is diabetic'}
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))




In [21]:
!ngrok config add-authtoken 1lvCYTrdRUcp7ZbgSrlXVhyJhqn_6iiS8N5byVMukLRCFHjsa

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:

# Set up ngrok tunnel
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

# Run the FastAPI server using uvicorn
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://644b-35-196-130-94.ngrok-free.app


INFO:     Started server process [150]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2401:4900:6763:36c3:57f:feeb:d6fa:ecae:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2401:4900:6763:36c3:57f:feeb:d6fa:ecae:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2401:4900:6763:36c3:57f:feeb:d6fa:ecae:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2401:4900:6763:36c3:57f:feeb:d6fa:ecae:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2401:4900:6763:36c3:57f:feeb:d6fa:ecae:0 - "POST /diabetes_prediction HTTP/1.1" 200 OK
